# 1. Importar tablas

In [1]:
timeorigin: .z.t

In [2]:
/ 1.1. Importar tabla empleados
employees:get `:data/employees;

/show employees;

In [3]:
/ 1.2. Importar tabla remuneraciones
remunerations:get `:data/remunerations;

/show employees;

In [4]:
/ 1.3. Importar tabla de bases mínimas y máximas
minMaxBasis:("DDFFI";enlist csv) 0: `$":data/minMaxBasis.csv";

/show minMaxBasis

# 2. Variables globales

In [5]:
argname:`commonBase,
        `profBase,
        `hExtraFMBase,
        `hExtraBase,
        `irpfBase,
        `meiBase;

argrems:(`sb`cs`ge`se; 
         `sb`cs`hefm`her`ge`se;
         `hefm;
         `her;
         `sb`cs`hefm`her`ge`se`pns;
         `sb`cs`ge`se);

argdate:0Nm,
        0Nm,
        0Nm,
        0Nm,
        0Nm,
        2023.01m;

# 3. Funciones


In [6]:
getremsum:{[name;lsim;sd]:() xkey select baseType:name, sum amount     /(get) (rem)uneration (sum)
                                  by nif, month 
                                  from remunerations 
                                  where remunerationType in lsim, month>=sd;}

minmax:{[n;cg;mo;bt]                                                   /values between (min)imumm & (max)imum
    if[bt=`profBase; cg:4];
    first exec maxBase & n | minBase 
    from minMaxBasis
    where contrGroup= 4&cg, mo within(startDate;endDate)};

# 4. Generación de las tablas

In [7]:
contributionBasis: (raze getremsum'[argname;argrems;argdate]) 
                    lj `nif xkey select contrGroup, nif from employees;

update amount: minmax'[amount;contrGroup;month;baseType] 
  from `contributionBasis 
  where baseType in `commonBase`profBase`meiBase;

delete contrGroup 
  from `contributionBasis;

In [8]:
timeend: .z.t

Si se quisiese ver de forma más visual de las bases de cotización se puede pivotar la tabla de la siguiente manera:

In [9]:
show 0^exec `commonBase`profBase`hExtraFMBase`hExtraBase`irpfBase`meiBase#(baseType!amount) by nif,month from contributionBasis;

nif       month  | commonBase profBase hExtraFMBase hExtraBase irpfBase meiBa..
-----------------| ----------------------------------------------------------..
00193031S 2021.07| 1050       1050     0            0          789.4722 0    ..
00193031S 2021.08| 2516.077   2516.077 0            0          2516.077 0    ..
00193031S 2021.09| 2726.691   2726.691 0            0          2726.691 0    ..
00193031S 2021.10| 2505.712   2505.712 0            0          2505.712 0    ..
00193031S 2022.07| 1166.7     1166.7   0            0          739.5401 0    ..
00193031S 2022.08| 2708.856   2708.856 0            0          2708.856 0    ..
00193031S 2022.09| 2727.476   2727.476 0            0          2727.476 0    ..
00193031S 2022.10| 2388.047   2464.806 0            76.75866   2464.806 0    ..
00193031S 2023.07| 1260       1260     0            0          794.9042 1260 ..
00193031S 2023.08| 2470.244   2470.244 0            0          2470.244 2470...
00193031S 2023.09| 2604.96    2604.96  0

# 5. Guardar datos en csv

In [10]:
`:data/contributionBasis set contributionBasis;

In [11]:
"Number of contribution basis: ", string count contributionBasis

"Total time spent: ",string timeend-timeorigin

"Number of contribution basis: 238730"


"Total time spent: 00:00:00.434"
